In [163]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
import numpy.linalg as LA
#np.set_printoptions(precision=3)
plt.rcParams['figure.figsize'] = (12.0, 8.0)

@np.vectorize
def roundsf(x, n):
     if n < 1:
         raise ValueError("number of significant digits must be >= 1")
     # Use %e format to get the n most significant digits, as a string.
     format = "%." + str(n-1) + "e"
     as_string = format % x
     return float(as_string)

In [169]:
PIN = 165
k1 = 1.0 + PIN / 300.0
k2 = k1 * 1e-3

#load the matrix M and the vector b
M = np.array([[k2, 3, 6,  7],
              [8, 2, 11, 6],
              [2, 9, 3,  1],
              [2, 4, 6,  1]])

b = np.array([5, 11, 3, 27])

truncate = 4

## Factorisation

In [170]:
def lu_factorisation(P):
    """
    compute P = LU where L is lower triangular and U is upper triangular
    returns L, U
    """
    m,n = P.shape
    assert(m == n) #this implementation assumes square matrices
    N = m
    
    L, U = np.zeros_like(P), np.zeros_like(P)
    
    #syntax notes:
    #range(a,b) returns [a, a+1, ... b-2, b-1]
    #arrays are zero referenced
    #U[i, j] follows the row, column convention
    
    #loop over the columns of L and U
    for j in range(N):
        for i in range(j+1):
            U[i, j] = P[i, j] - sum(L[i, k]*U[k, j] for k in range(i))
            
        for i in range(j+1, N): 
            L[i, j] = P[i, j] - sum(L[i, k]*U[k, j] for k in range(j))
            L[i, j] /= U[j,j]
        
        #explictely set the diagonal of L to 1
        L[j, j] = 1
    
    #round the returned matrices to 3 d.p
    if truncate: return roundsf(L, truncate), roundsf(U, truncate)
    else: return L, U
            

        
    
L, U = lu_factorisation(M)

print("L = \n", L, "\n")
print("U = \n", U, "\n")
print("LU = \n", np.dot(L,U), "\n") #in numpy, matrix multiplication is np.dot(A,B)
print("M = \n", M, "\n")

L = 
 [[  1.0000000000e+00   0.0000000000e+00   0.0000000000e+00
    0.0000000000e+00]
 [  5.1610000000e+03   1.0000000000e+00   0.0000000000e+00
    0.0000000000e+00]
 [  1.2900000000e+03   2.4950000000e-01   1.0000000000e+00
    0.0000000000e+00]
 [  1.2900000000e+03   2.4980000000e-01   2.2380000000e-01
    1.0000000000e+00]] 

U = 
 [[  1.5500000000e-03   3.0000000000e+00   6.0000000000e+00
    7.0000000000e+00]
 [  0.0000000000e+00  -1.5480000000e+04  -3.0960000000e+04
   -3.6120000000e+04]
 [  0.0000000000e+00   0.0000000000e+00  -1.6750000000e+01
   -2.0330000000e+01]
 [  0.0000000000e+00   0.0000000000e+00   0.0000000000e+00
   -4.1150000000e+00]] 

LU = 
 [[  1.5500000000e-03   3.0000000000e+00   6.0000000000e+00
    7.0000000000e+00]
 [  7.9995500000e+00   3.0000000000e+00   6.0000000000e+00
    7.0000000000e+00]
 [  1.9995000000e+00   7.7400000000e+00  -1.2700000000e+00
   -2.2700000000e+00]
 [  1.9995000000e+00   3.0960000000e+00   2.4433500000e+00
   -1.4408540000e+00]] 



## Forward Subsitution

In [171]:
def LU_solve(L, U, b):
    """
    solve LUx = b
    for x where L and U are lower and upper triangular
    """
    N = len(b)
    y = np.zeros((N))
    x = np.zeros((N))
    
    #first step: solve for y in Ly = b
    for i in range(N):
        y[i] = b[i] - sum(L[i,k] * y[k] for k in range(i))
    
    if truncate: y = roundsf(y, truncate)
    #second step: solve for x in Ux = y, note that the order of evaluation is reversed with [::-1]
    for i in range(N)[::-1]:
        x[i] = ( y[i] - sum(U[i,k] * x[k] for k in range(i+1, N)) ) / U[i,i]
    
    if truncate: x = roundsf(x, truncate)
    return x

x = LU_solve(L, U, b)
exact_x = LA.solve(M, b)

print("x from LU factorisation = ", x)
print("exact x = ", exact_x)
print("% error = ", 100 * (x - exact_x) / x, "\n")

print("Mx = ",np.dot(M, x))
print("b = ", b)
    

x from LU factorisation =  [ 1.25   -0.2188  7.432  -5.563 ]
exact x =  [-4.456057567  -0.4896717759  7.1838803735 -5.2324800034]
% error =  [ 456.4846053612 -123.798800683     3.3385310346    5.941398464 ] 

Mx =  [  4.9965375  57.9364     17.2638     40.6538   ]
b =  [ 5 11  3 27]


In [32]:
def inverse(P):
    #get the shape of P
    #we're going to assume it's square
    n,m = P.shape
    #get the LU factorisation of M
    L, U = lu_factorisation(M)
    #generate an identity matrix of the correct size
    I = np.identity(n)
    #allocate space for the inverse matrix
    inv_P = np.zeros_like(P)
    #iterate over the columns of I (which are also the rows since I is hermitian)
    for i in range(n):
        inv_P[:, i] = LU_solve(L, U, I[i])
    return inv_P
        

inv_M = inverse(M)
print("calculated inverse using my method = \n", inv_M, "\n")
from numpy.linalg import inv
print("calculated inverse using a library method = \n", inv(M), "\n")
print("difference = \n", inv(M) - inv_M, "\n")
print("inv_M * M = \n", np.dot(inv_M,M), "\n")

    

calculated inverse using my method = 
 [[-0.14065319  0.18778306  0.12297949 -0.26510549]
 [ 0.00883685 -0.02750469  0.12316359 -0.01999337]
 [ 0.01472808 -0.04584116 -0.12806068  0.30001105]
 [ 0.15759049  0.00949961  0.02975073 -0.18988181]] 

calculated inverse using a library method = 
 [[-0.14065319  0.18778306  0.12297949 -0.26510549]
 [ 0.00883685 -0.02750469  0.12316359 -0.01999337]
 [ 0.01472808 -0.04584116 -0.12806068  0.30001105]
 [ 0.15759049  0.00949961  0.02975073 -0.18988181]] 

difference = 
 [[ -5.55111512e-17   2.77555756e-17   5.55111512e-17  -5.55111512e-17]
 [ -1.38777878e-17   0.00000000e+00  -1.38777878e-17   3.46944695e-17]
 [  4.85722573e-17  -6.93889390e-18   0.00000000e+00   0.00000000e+00]
 [ -2.77555756e-17  -3.46944695e-18  -1.04083409e-17   5.55111512e-17]] 

inv_M * M = 
 [[  1.00000000e+00  -3.33066907e-16  -2.22044605e-16   2.22044605e-16]
 [  0.00000000e+00   1.00000000e+00  -5.55111512e-17   8.32667268e-17]
 [  0.00000000e+00  -3.46944695e-17   1.000

In [61]:
from operator import mul
from functools import reduce
def product(nums):
    return reduce(mul, nums)

def determinant(P):
    L,U = lu_factorisation(P)
    n,m = P.shape
    return product(U[i][i] for i in range(n))

print("Determinant from library function = ", LA.det(M), "\n")
print("Determinant from the product of the diagonal of U = ", determinant(M))
    

Determinant from library function =  -1357.95 

Determinant from the product of the diagonal of U =  -1357.95


## Tridiagonal Matrices

In [181]:
PIN1 = 165
k3 = 1 + PIN1 / 300
k4 = k3 * 1e-5
#looking at the matrix equation Tx = b where x in unknown
b = np.arange(1,10)

#using k = k0 = k
gamma = np.ones_like(b) * -k3
beta = np.ones_like(b) * (1 + 2*k3)
beta[0] = k4
alpha = np.ones_like(b) * -k3

def lu_factor_tridiagonal(alpha, beta, gamma): #alpha is the lower diagonal entries, beta the central and gamma the upper 
    N = len(beta)
    d, l, u = np.zeros((3, N))

    #note that the first and last iterations of the loop are written explicitely to avoid out of bounds access
    d[0] = beta[0]
    l[1] = alpha[1] / d[0]
    
    for j in range(1, N-1):
        u[j - 1] = gamma[j - 1]
        d[j] = beta[j] - l[j]*u[j-1]
        l[j+1] = alpha[j+1] / d[j]
    
    u[N - 2] = gamma[N - 2]
    d[N-1] = beta[N-1] - l[N-1]*u[N-2]
        
    if truncate: return roundsf(d,truncate), roundsf(l, truncate), roundsf(u, truncate)       
    return d,l,u

def lu_solve_tridiag(d,l,u,b):
    N = len(b)
    x,y = np.zeros((2, N))
    
    #forward substitution
    y[0] = b[0]
    for j in range(1, N):
        y[j] = b[j] - l[j]*y[j-1]
    
    #backward substitution
    x[N-1] = y[N-1] / d[N-1]
    for j in range(N-1)[::-1]:
        x[j] = (y[j] - u[j] * x[j+1]) / d[j]
        
    if truncate: return roundsf(x, truncate)
    return x
        
def bcalc(alpha, beta, gamma ,x):
    N = len(x)
    b = np.zeros(N)
    b[0] = beta[0]*x[0] + gamma[0] * x[1]
    for i in range(1,N-1):
        b[i] = alpha[i]*x[i-1] + beta[i]*x[i] + gamma[i] * x[i+1]
    
    b[N-1] = alpha[N-1]*x[N-2] + beta[N-1]*x[N-1]   
    
    if truncate: return roundsf(b, truncate)
    return b
  
truncate = 4
d,l,u = lu_factor_tridiagonal(alpha, beta, gamma)
x = lu_solve_tridiag(d,l,u,b)

np.set_printoptions(precision=4)
print("Solving Tx = (1,2,3,4,5,6,7,8,9)\n 4 sig figs")

print("alpha: ", alpha)
print("beta: ", beta)
print("gamma: ", gamma, "\n")

print("d: ", d)
print("l: ", l)
print("u: ", u, "\n")

print("x: ", x)
print("Tx", bcalc(alpha, beta, gamma, x))
print("original b: ",b)
print("max % error: {:.2}".format(max(abs(b - bcalc(alpha, beta, gamma, x)) / b * 100 )))

Solving Tx = (1,2,3,4,5,6,7,8,9)
 4 sig figs
alpha:  [-1.55 -1.55 -1.55 -1.55 -1.55 -1.55 -1.55 -1.55 -1.55]
beta:  [  1.5500e-05   4.1000e+00   4.1000e+00   4.1000e+00   4.1000e+00
   4.1000e+00   4.1000e+00   4.1000e+00   4.1000e+00]
gamma:  [-1.55 -1.55 -1.55 -1.55 -1.55 -1.55 -1.55 -1.55 -1.55] 

d:  [  1.5500e-05  -1.5500e+05   4.1000e+00   3.5140e+00   3.4160e+00
   3.3970e+00   3.3930e+00   3.3920e+00   3.3920e+00]
l:  [  0.0000e+00  -1.0000e+05   1.0000e-05  -3.7800e-01  -4.4110e-01
  -4.5370e-01  -4.5630e-01  -4.5690e-01  -4.5700e-01]
u:  [-1.55 -1.55 -1.55 -1.55 -1.55 -1.55 -1.55 -1.55  0.  ] 

x:  [-3.049  -0.6452  1.758   3.36    4.55    5.449   5.993   5.888   4.421 ]
Tx [ 1.     -0.6443  3.      3.999   5.001   5.999   6.999   7.999   9.    ]
original b:  [1 2 3 4 5 6 7 8 9]
max % error: 1.3e+02


-5.56e-03
1.22e+00


array([-0.00556,  1.22   ])

10.0